# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date,title,venue,excerpt,citation,url_slug,paper_url
2020-01-01,Inferring layer-by-layer composition in Au-Ag nanoparticles using a combination of X-ray Photoelectron Spectroscopy and Monte Carlo simulations,Surface Science,hi,Arif-2020-inferring,Arif-2020-inferring,https://doi.org/10.1016/j.susc.2019.121503
2023-12-23,Defect Engineering of WO3 by Rapid Flame Reduction for Efficient Photoelectrochemical Conversion of Methane into Liquid Oxygenates,Nano Letters,hi,Woo-2023-wo3-ch4-oxidation,Woo-2023-wo3-ch4-oxidation,https://doi.org/10.1021/acs.nanolett.3c03131
2024-05-15,Implication of surface oxidation of nanoscale molybdenum carbide on electrocatalytic activity,Journal of Materials Chemistry A,hi,Yu-2024-beta-mo2c,Yu-2024-beta-mo2c,https://doi.org/10.1039/D4TA01746C
2025-04-01,Role of surface oxygen in α-MoC catalyst stability and activity under electrooxidation conditions,ACS Catalysis,hi,Gautam-2025-amoc-faeo,Gautam-2025-amoc-faeo,https://doi.org/10.1021/acscatal.4c06544
2025-0

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep=",", header=0)
publications

,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2020-01-01,Inferring layer-by-layer composition in Au-Ag ...,Surface Science,hi,Arif-2020-inferring,Arif-2020-inferring,https://doi.org/10.1016/j.susc.2019.121503
1,2023-12-23,Defect Engineering of WO3 by Rapid Flame Reduc...,Nano Letters,hi,Woo-2023-wo3-ch4-oxidation,Woo-2023-wo3-ch4-oxidation,https://doi.org/10.1021/acs.nanolett.3c03131
2,2024-05-15,Implication of surface oxidation of nanoscale ...,Journal of Materials Chemistry A,hi,Yu-2024-beta-mo2c,Yu-2024-beta-mo2c,https://doi.org/10.1039/D4TA01746C
3,2025-04-01,Role of surface oxygen in α-MoC catalyst stabi...,ACS Catalysis,hi,Gautam-2025-amoc-faeo,Gautam-2025-amoc-faeo,https://doi.org/10.1021/acscatal.4c06544
4,2025-05-01,Planar chiral metallopolymers for electrochemi...,Journal of the American Chemical Society,hi,Jeon-2025-planar-selectivity,Jeon-2025-planar-selectivity,https://doi.org/10.1021/jacs.5c01571
5,2025-06-01,Elucidating the electronic effects of substitu...,in preparation,hi,Gautam-2025-nhc-stability,Gautam-2025-nhc-stability,_
6,2025-07-01,Developing minimally-empirical tight-binding m...,in preparation,hi,Gautam-2025-ntb-la-h2o,Gautam-2025-ntb-la-h2o,_


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2020-01-01-Arif-2020-inferring.md
2023-12-23-Woo-2023-wo3-ch4-oxidation.md
2024-05-15-Yu-2024-beta-mo2c.md
2025-04-01-Gautam-2025-amoc-faeo.md
2025-05-01-Jeon-2025-planar-selectivity.md
2025-06-01-Gautam-2025-nhc-stability.md
2025-07-01-Gautam-2025-ntb-la-h2o.md


In [7]:
!cat ../_publications/2020-01-01-arif-2020-inferring.md

---
title: "Inferring layer-by-layer composition in Au-Ag nanoparticles using a combination of X-ray Photoelectron Spectroscopy and Monte Carlo simulations"
collection: publications
permalink: /publication/2020-01-01-Arif-2020-inferring
date: 2020-01-01
venue: 'Surface Science'
paperurl: 'https://doi.org/10.1016/j.susc.2019.121503'
citation: 'Arif-2020-inferring'
---
[Download paper here](https://doi.org/10.1016/j.susc.2019.121503)

Recommended citation: Arif-2020-inferring